In [1]:
import numpy as np
import tensorflow as tf

In [2]:
n_inputs = 3
n_neurons = 5

In [3]:
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

In [4]:
Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

Instructions for updating:
Colocations handled automatically by placer.


## 1. 아주 low 단계에서 RNN 설계

In [5]:
# model
# Y = Wx * X + b
# 

Y0 = tf.tanh(tf.matmul(X0, Wx) + b) # step 1일 때 RNN
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) +  b) # step 2일 때 RNN

In [6]:
X0_batch = np.array([[0, 1, 2],
                   [3, 4, 5],
                   [6, 7, 8],
                   [9, 0, 1]])

X1_batch = np.array([[9, 8, 7],
                   [3, 4, 5],
                   [6, 5, 4],
                   [3, 2, 1]])

In [7]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    Y0_var, Y1_var = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})
    
    print('Y0_val: {} \n {}'.format(Y0_var.shape, Y0_var))
    print('Y1_val: {} \n {}'.format(Y1_var.shape, Y1_var))

Y0_val: (4, 5) 
 [[ 0.7546629  -0.73342264  0.77572215 -0.9451782  -0.95958626]
 [ 0.8984897   0.96359855  0.9995419  -0.9996191  -0.99999815]
 [ 0.95992494  0.9998943   0.99999917 -0.99999756 -1.        ]
 [-1.          1.         -0.99872357  0.67569095  0.99999774]]
Y1_val: (4, 5) 
 [[ 0.42492905  1.          0.9999998  -1.         -1.        ]
 [ 0.95750326  0.98144305  0.9998393  -0.9999956  -0.99998254]
 [ 0.4150279   0.99999887  0.99998397 -0.99996287 -0.9999979 ]
 [ 0.09465872  0.9997159   0.9582563   0.99814814 -0.98945314]]


## 2. tf.nn.rnn_cell.BasicRNNCell() + tf.nn.static_rnn()

In [8]:
# model

# X0 -> (batch_size, input_size) step 1
# X1 -> (batch_size, input_size) step 2
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)

output_seqs, states = tf.nn.static_rnn(cell=basic_cell, inputs=[X0, X1], dtype=tf.float32)
Y0, Y1 = output_seqs

# time step = 0 (4, 3)
X0_batch = np.array([[0, 1, 2],
                   [3, 4, 5],
                   [6, 7, 8],
                   [9, 0, 1]])

# time step = 1 (4, 3)
X1_batch = np.array([[9, 8, 7],
                   [3, 4, 5],
                   [6, 5, 4],
                   [3, 2, 1]])

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    Y0_val, Y1_val = sess.run([Y0, Y1], {X0: X0_batch, X1: X1_batch})
    
    print('Y0_val:{}\n{}'.format(Y0_val.shape, Y0_val))

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Y0_val:(4, 5)
[[-0.8837547  -0.21412787 -0.07898846 -0.846175    0.1161041 ]
 [-0.996454    0.50743496 -0.950489   -0.9994738   0.45585933]
 [-0.99989784  0.870716   -0.99849135 -0.9999982   0.70010877]
 [ 0.85509706  0.9999933  -0.9992286  -0.99943894  0.50988954]]


- tf.nn.rnn_cell.BasicRNNCell()
  - 일반적인 RNNCell
  - GPU optim한 모듈은 tf.contrib.cudnn_rnn.CudnnRNNTanh 사용
  - num_units : RNN cell의 unit 수
  - activation : nonlinearity (Tanh)

- tf.nn.static_rnn()
  - RNNCell을 이용해 reccurent neural network 생성
  -> state = cell.zero_state()
  -> outputs = []
  -> for input_ in inputs:
        output, state = cell(input_, state)
        outputs.append(output)
     return (outputs, state)
  - input으로 step input과 state
  - output으로 step별 output과 마지막 state 출력
  
  - cell: RNNCell
  - inputs: input list [batch_size, input]의 리스트로 저장 -> [seq_len, batch_size, input]
  - initial_state: 초기 initial 값 
  - dtype: state, output의 data type를 지정
  - sequence_length : sequence의 길이를 입력

  - 단점 : 타임 스텝마다 하나의 셀을 추가하는 방식 -> 스텝이 많을 경우 OOM 발생

## 3. tf.nn.rnn_cell.BasicRNNCell() + tf.nn.dynamic_rnn()

- tf.nn.dynamic_rnn()
  - RNNCell을 이용해 recurrent neural network 수행
  - cell : RNNCell instance
  - inputs : time_major is False -> [batch_size, max_time, input_size]
             time_major is True -> [max_time, batch_size, input_size]
  - sequence_length : 각 batch의 길이 [batch_size]
  - dtype: date type
  - time_major : input, output shape 선택 (True일 때, 더 효휼적 transpose를 피할 수 있다.) * [max_time, batch_size, input_size]

In [9]:
# model

n_steps = 2
n_inputs = 3
n_hidden = 5

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

basic_cell2 = tf.nn.rnn_cell.BasicRNNCell(num_units=n_hidden)
outputs, states = tf.nn.dynamic_rnn(basic_cell2, inputs=X, dtype=tf.float32)

# (batch_size, time_len, input_size)
X_batch = np.array([
    [[0, 1, 2], [9, 8, 7]],
    [[3, 4, 5], [3, 4, 5]],
    [[6, 7, 8], [6, 5, 4]],
    [[9, 0, 1], [3, 2, 1]]]) 

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    outputs_val = outputs.eval(feed_dict={X: X_batch}) 
    print('outputs_val:{}\n{}'.format(outputs_val.shape, outputs_val))

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


ValueError: Variable rnn/basic_rnn_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-8-518bd1577459>", line 10, in <module>
    output_seqs, states = tf.nn.static_rnn(cell=basic_cell, inputs=[X0, X1], dtype=tf.float32)
  File "/usr/local/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/usr/local/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3248, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):


## tf.nn.static_rnn() vs tf.nn.dynamic_rnn() 차이
- tf.nn.static_rnn : 
  - fixed RNN length을 사용 -> 200 step시 200개의 rnn 생성, graph 생성이 느림
  - 200 step 이상 사용 불가
  
- tf.nn.dynamic_rnn()
  - tf.while 사용하여 생성, graph 생성이 빠름
  - 다양한 사이즈의 batch feed_dict 가능